In [1]:
import os

In [2]:
os.environ["SPARK_HOME"]="/home/bof/Documents/PROJET/spark-3.5.0-bin-hadoop3"
os.environ["JAVA_HOME"]="/usr/lib/jvm/default-java"

In [3]:
import pyspark
import psycopg2
import threading
import requests
import json
import time
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_unixtime, date_format, hour, dayofmonth
from pyspark.sql.types import FloatType

In [4]:
# Connect to the PostgreSQL database
host = "172.17.0.2"
port = 5432
user = "postgres"
database = "db_atmo"
password = "mysecretpassword"

conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)

cur = conn.cursor()


In [5]:
import psycopg2
from tabulate import tabulate

# Paramètres de connexion
host = "172.17.0.2"
port = 5432
user = "postgres"
database = "db_atmo"
password = "mysecretpassword"

# Établir la connexion
conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)

# Créer un curseur
cur = conn.cursor()

# Exécuter une requête SQL avec GROUP BY pour obtenir les résultats agrégés
cur.execute("SELECT code_zone, COUNT(*) FROM episodes_de_pollution_constatés_sur_l’année_passée GROUP BY code_zone")

# Récupérer toutes les lignes
rows = cur.fetchall()

# Fermer le curseur et la connexion
cur.close()
conn.close()

# Afficher les résultats sous forme de tableau
print(tabulate(rows, headers=['Code Zone', 'Nombre de lignes'], tablefmt='grid'))


+-------------+--------------------+
| Code Zone   |   Nombre de lignes |
+=============+====================+
| 249730037   |                 56 |
+-------------+--------------------+
| 13          |                151 |
+-------------+--------------------+
| 28          |                 23 |
+-------------+--------------------+
| 2008        |                 34 |
+-------------+--------------------+
| 39          |                  5 |
+-------------+--------------------+
| 14          |                  3 |
+-------------+--------------------+
| 25          |                  5 |
+-------------+--------------------+
| 2012        |                 12 |
+-------------+--------------------+
| 34          |                  9 |
+-------------+--------------------+
| 64          |                  5 |
+-------------+--------------------+
| 79          |                  3 |
+-------------+--------------------+
| 18          |                 14 |
+-------------+--------------------+
|

In [5]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


# Créer une instance de SparkSession avec les propriétés de configuration
spark = SparkSession.builder.appName("Données ATMO").config("spark.driver.extraClassPath", "/home/bof/Documents/PROJET/spark-3.5.0-bin-hadoop3/jars/postgresql-42.6.2.jar").config("spark.executor.extraClassPath", "/home/bof/Documents/PROJET/spark-3.5.0-bin-hadoop3/jars/postgresql-42.6.2.jar").getOrCreate()



24/03/26 10:03:44 WARN Utils: Your hostname, hp resolves to a loopback address: 127.0.1.1; using 192.168.0.108 instead (on interface wlp1s0)
24/03/26 10:03:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/26 10:03:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/26 10:03:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:

# Charger les données depuis PostgreSQL
df_indice_atmo = spark.read.format("jdbc").option("url", "jdbc:postgresql://172.17.0.2:5432/db_atmo").option("dbtable", "indice_atmo").option("user", "postgres").option("password", "mysecretpassword").load()

# Charger les données depuis PostgreSQL
df_region = spark.read.format("jdbc").option("url", "jdbc:postgresql://172.17.0.2:5432/db_atmo").option("dbtable", "region").option("user", "postgres").option("password", "mysecretpassword").load()


# Charger les données depuis PostgreSQL
df_epci = spark.read.format("jdbc").option("url", "jdbc:postgresql://172.17.0.2:5432/db_atmo").option("dbtable", "epci").option("user", "postgres").option("password", "mysecretpassword").load()


# Charger les données depuis PostgreSQL
df_annee = spark.read.format("jdbc").option("url", "jdbc:postgresql://172.17.0.2:5432/db_atmo").option("dbtable", "episodes_de_pollution_constatés_sur_l’année_passée").option("user", "postgres").option("password", "mysecretpassword").load()




In [7]:
# Convertir le DataFrame Spark en DataFrame Pandas
df_i = df_indice_atmo.toPandas()
df_r = df_region.toPandas()
df_e = df_epci.toPandas()
df_a = df_annee.toPandas()

In [10]:
df_i

id  code_no2  code_o3  code_pm10  code_pm25  code_qual  code_so2  \
0        1         2        1          2          3          3         1   
1        2         1        1          2          2          2         1   
2        3         1        2          1          1          2         1   
3        4         1        1          1          1          1         1   
4        5         2        1          1          2          2         1   
...    ...       ...      ...        ...        ...        ...       ...   
2035  2036         2        2          2          2          2         1   
2036  2037         1        2          1          1          2         1   
2037  2038         1        2          1          1          2         1   
2038  2039         1        2          1          1          2         1   
2039  2040         1        2          1          1          2         1   

     code_zone coul_qual    date_dif    date_ech epsg_reg lib_qual lib_zone  \
0        26362   #f0e641  2022-01-04  2022-01-03     2154  Dégradé  Valence   
1        26362   #50ccaa  2022-01-05  2022-01-04     2154    Moyen  Valence   
2        26362   #50ccaa  2022-01-06  2022-01-05     2154    Moyen  Valence   
3        26362   #50f0e6  2022-01-07  2022-01-06     2154      Bon  Valence   
4        26362   #50ccaa  2022-01-08  2022-01-07     2154    Moyen  Valence   
...        ...       ...         ...         ...      ...      ...      ...   
2035     26362   #50ccaa  2024-02-15  2024-02-15     2154    Moyen  Valence   
2036     26362   #50ccaa  2024-02-18  2024-02-17     2154    Moyen  Valence   
2037     26362   #50ccaa  2024-02-18  2024-02-18     2154    Moyen  Valence   
2038     26362   #50ccaa  2024-02-18  2024-02-19     2154    Moyen  Valence   
2039     26362   #50ccaa  2024-02-18  2024-02-20     2154    Moyen  Valence   

                         source type_zone                      x_reg  \
0     Atmo Auvergne-Rhône-Alpes   commune  849260.200000000000000000   
1     Atmo Auvergne-Rhône-Alpes   commune  849260.200000000000000000   
2     Atmo Auvergne-Rhône-Alpes   commune  849260.200000000000000000   
3     Atmo Auvergne-Rhône-Alpes   commune  849260.200000000000000000   
4     Atmo Auvergne-Rhône-Alpes   commune  849260.200000000000000000   
...                         ...       ...                        ...   
2035  Atmo Auvergne-Rhône-Alpes   commune  849260.200000000000000000   
2036  Atmo Auvergne-Rhône-Alpes   commune  849260.200000000000000000   
2037  Atmo Auvergne-Rhône-Alpes   commune  849260.200000000000000000   
2038  Atmo Auvergne-Rhône-Alpes   commune  849260.200000000000000000   
2039  Atmo Auvergne-Rhône-Alpes   commune  849260.200000000000000000   

                   x_wgs84                       y_reg                y_wgs84  
0     4.892107529616570000  6427795.900000000000000000  44.933269680647700000  
1     4.892107529616570000  6427795.900000000000000000  44.933269680647700000  
2     4.892107529616570000  6427795.900000000000000000  44.933269680647700000  
3     4.892107529616570000  6427795.900000000000000000  44.933269680647700000  
4     4.892107529616570000  6427795.900000000000000000  44.933269680647700000  
...                    ...                         ...                    ...  
2035  4.892107529616560000  6427795.900000000000000000  44.933269680647800000  
2036  4.892107529616560000  6427795.900000000000000000  44.933269680647800000  
2037  4.892107529616560000  6427795.900000000000000000  44.933269680647800000  
2038  4.892107529616560000  6427795.900000000000000000  44.933269680647800000  
2039  4.892107529616560000  6427795.900000000000000000  44.933269680647800000  

[2040 rows x 20 columns]

In [11]:
df_r.head(10)

id                superficie                  population  \
0   1   1128.000000000000000000   386875.000000000000000000   
1   2   1128.000000000000000000   386875.000000000000000000   
2   3   1128.000000000000000000   386875.000000000000000000   
3   4   1128.000000000000000000   386875.000000000000000000   
4   5   1128.000000000000000000   386875.000000000000000000   
5   6   1128.000000000000000000   386875.000000000000000000   
6   7  85053.141063500000000000  5985571.000000000000000000   
7   8  85053.141063500000000000  5985571.000000000000000000   
8   9  85053.141063500000000000  5985571.000000000000000000   
9  10  85053.141063500000000000  5985571.000000000000000000   

                       pm25                      pm10  \
0    493.198255617000000000    779.220113707000000000   
1     57.983897469000000000     63.142489586000000000   
2    158.890851610000000000    250.614599361000000000   
3     16.977230564000000000     40.342513575000000000   
4     50.412483695000000000     52.873560605000000000   
5     43.432949459000000000    135.541123501000000000   
6  16887.889908878000000000  30354.395237306000000000   
7    180.607067693000000000    361.998992045000000000   
8   2337.373797801000000000   3290.261239414000000000   
9   2852.023591328000000000  12023.322129537000000000   

                        nox code_pcaet                          ges code  
0   6468.636929901000000000        All   2332639.820023450000000000   02  
1    980.398021830000000000          7     65482.522177877000000000   02  
2   2145.371134360000000000          6    685960.242005027000000000   02  
3    158.069196401000000000          5     97602.475428539000000000   02  
4     47.334382444000000000         34     36980.742838358000000000   02  
5     97.965068604000000000          2    158412.924896654000000000   02  
6  91467.396539170000000000        All  40228293.067131900000000000   75  
7   3270.330248279000000000          7    166777.086640953000000000   75  
8  48318.253282295000000000          6  15293724.941467500000000000   75  
9  20168.508819889000000000          5  10394694.544878400000000000   75

In [12]:
df_e.head(10)

id              superficie                 population  \
0  4179   69.226750000000000000   50922.000000000000000000   
1  4180  450.178577900000000000   23027.000000000000000000   
2  4181  283.000000000000000000    5188.000000000000000000   
3  4182  330.932678500000000000   43989.000000000000000000   
4  4183  492.110000000000000000    8519.000000000000000000   
5  4184  244.520824000000000000  114149.000000000000000000   
6  4185  932.000000000000000000  130952.000000000000000000   
7  4186  346.979484500000000000   30213.000000000000000000   
8  4187  422.465022000000000000   13953.000000000000000000   
9  4188  356.034875500000000000   47525.000000000000000000   

                     pm25                    pm10                      nox  \
0   68.090000000000000000  100.080000000000000000   362.600000000000000000   
1   82.260000000000000000  141.870000000000000000   411.700000000000000000   
2   10.980000000000000000   15.730000000000000000    65.630000000000000000   
3  103.880000000000000000  133.700000000000000000   552.410000000000000000   
4   78.550000000000000000  120.640000000000000000   292.820000000000000000   
5  155.730000000000000000  225.190000000000000000  1111.830000000000000000   
6  222.000000000000000000  407.940000000000000000  2932.830000000000000000   
7  117.710000000000000000  241.430000000000000000   733.260000000000000000   
8   83.030000000000000000  207.250000000000000000   386.760000000000000000   
9  112.170000000000000000  193.200000000000000000   451.830000000000000000   

                          ges code_pcaet       code  
0   217928.630000000000000000        All  247700339  
1   352864.030000000000000000        All  200042729  
2    39651.830000000000000000        All  200006930  
3   210264.160000000000000000        All  243301264  
4   133112.730000000000000000        All  200036150  
5   481847.890000000000000000        All  246800726  
6  1179424.330000000000000000        All  241100593  
7   279527.840000000000000000        All  200041994  
8   153376.870000000000000000        All  200043016  
9   221002.680000000000000000        All  200023307

In [13]:
df_a.head(50)

id          lib_pol       lib_zone                           etat  \
0    1            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
1    2            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
2    3            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
3    4            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
4    5            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
5    6            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
6    7  Dioxyde d'azote  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
7    8            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
8    9            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
9   10            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
10  11            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
11  12  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
12  13  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
13  14  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
14  15  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
15  16            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
16  17            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
17  18            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
18  19            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
19  20  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
20  21            Ozone  ILE-DE-FRANCE                         ALERTE   
21  22            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
22  23            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
23  24  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
24  25  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
25  26  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
26  27  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
27  28  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
28  29  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
29  30  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
30  31  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
31  32            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
32  33            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
33  34            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
34  35            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
35  36            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
36  37            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
37  38            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
38  39            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
39  40            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
40  41  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
41  42  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
42  43  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
43  44  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
44  45  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
45  46  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
46  47            Ozone  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
47  48  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
48  49  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   
49  50  Particules PM10  ILE-DE-FRANCE  INFORMATION ET RECOMMANDATION   

      date_ech    date_dif code_zone code_pol  
0   2018-07-07  2018-07-06        11        7  
1   2018-07-08  2018-07-07        11        7  
2   2018-07-16  2018-07-15        11        7  
3   2018-07-23  2018-07-22        11        7  
4   2018-07-24  2018-07-23        11    

In [14]:
#visualiser des données nulles
null_counts = df_indice_atmo.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_indice_atmo.columns])

# Afficher les résultats
null_counts.show()

+---+--------+-------+---------+---------+---------+--------+---------+---------+--------+--------+--------+--------+--------+------+---------+-----+-------+-----+-------+
| id|code_no2|code_o3|code_pm10|code_pm25|code_qual|code_so2|code_zone|coul_qual|date_dif|date_ech|epsg_reg|lib_qual|lib_zone|source|type_zone|x_reg|x_wgs84|y_reg|y_wgs84|
+---+--------+-------+---------+---------+---------+--------+---------+---------+--------+--------+--------+--------+--------+------+---------+-----+-------+-----+-------+
|  0|       0|      0|        0|        0|        0|       0|        0|        0|       0|       0|       0|       0|       0|     0|        0|    0|      0|    0|      0|
+---+--------+-------+---------+---------+---------+--------+---------+---------+--------+--------+--------+--------+--------+------+---------+-----+-------+-----+-------+



In [9]:
#visualiser des données nulles
null_counts1 = df_region.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_region.columns])

# Afficher les résultats
null_counts1.show()

+---+----------+----------+----+----+---+----------+---+----+
| id|superficie|population|pm25|pm10|nox|code_pcaet|ges|code|
+---+----------+----------+----+----+---+----------+---+----+
|  0|         0|         0|   0|   0|  0|         0|  0|   0|
+---+----------+----------+----+----+---+----------+---+----+



In [10]:
#visualiser des données nulles
null_counts2 = df_epci.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_epci.columns])

# Afficher les résultats
null_counts2.show()

+---+----------+----------+----+----+---+---+----------+----+
| id|superficie|population|pm25|pm10|nox|ges|code_pcaet|code|
+---+----------+----------+----+----+---+---+----------+----+
|  0|         0|         0|   0|   0|  0|  0|         0|   0|
+---+----------+----------+----+----+---+---+----------+----+



In [11]:
#visualiser des données nulles
null_counts3 = df_annee.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_annee.columns])

# Afficher les résultats
null_counts3.show()

+---+-------+--------+----+--------+--------+---------+--------+
| id|lib_pol|lib_zone|etat|date_ech|date_dif|code_zone|code_pol|
+---+-------+--------+----+--------+--------+---------+--------+
|  0|      0|       0|   0|       0|       0|        0|       0|
+---+-------+--------+----+--------+--------+---------+--------+

